In [1]:
import pandas as pd
import numpy as np
import random
from sklearn import preprocessing
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, CuDNNLSTM, BatchNormalization
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint

from collections import deque
import time

In [6]:
df = pd.read_csv("GDAX-LTC_USD.csv",names=["date","open","high","low","volume"],skiprows=1,nrows=60)


,date,open,high,low,volume
55,2018-10-01,61.29,61.80,59.22,143529.403608
56,2018-09-24,61.51,61.78,57.07,175256.279532
57,2018-09-20,54.06,56.56,53.37,117563.416715
58,2018-09-18,51.81,55.49,50.55,171844.677173
59,2018-09-15,57.59,58.73,55.55,126008.450897


In [3]:
SEQ_LEN = 60 #data from the last 60 days
FUT_PERIOD_PREDICT = 3 #days into the future
RATIO_PREDICT = "LTC-USD"
EPOCHS = 10
BATCH_SIZE = 64
NAME = f"{SEQ_LEN}-SEQ-{FUT_PERIOD_PREDICT}-PRED-{int(time.time())}"


def classify(current,future):
    if float(future) > float(current):
        return 1 #future price is higher than current price. Good for buying
    else:
        return 0
    
def df_preprocess(df):
    df = df.drop("future",1)
    for column in df.columns:
        if column != "target":
            df[column] = df[column].pct_change()
            df.dropna(inplace=True)
            df[column] = preprocessing.scale(df[column].values)
            
    df.dropna(inplace=True)
    
    sequential_data = []
    previous_days = deque(maxlen=SEQ_LEN)
    
    for i in df.values:
        previous_days.append([n for n in i[:-1]])
        if len(previous_days) == SEQ_LEN:
            sequential_data.append([np.array(previous_days), i[-1]])
            
    random.shuffle(sequential_data)
    
    buys = []
    sells = []
    
    for seq, target in sequential_data:
        if target == 0:
            sells.append([seq,target])
        elif target == 1:
            buys.append([seq,target])
    
    random.shuffle(buys)
    random.shuffle(sells)
    
    lower = min(len(buys), len(sells))
    
    buys = buys[:lower]
    sells = sells[:lower]
    
    sequential_data = buys + sells
    random.shuffle(sequential_data)
    
    x = []
    y = []
    
    for seq, target in sequential_data:
        x.append(seq)
        y.append(target)
        
    return np.array(x), y
    
    


In [4]:
master_df = pd.DataFrame()
cryptos = ["BTC-USD","LTC-USD","ETH-USD","BCH-USD"]
for crypto in cryptos:
    dataset = f"{crypto}.csv"
    df = pd.read_csv(dataset, names=["time","low","high","open","close","volume"])
    df.rename(columns={"close": f"{crypto}_close", "volume": f"{crypto}_volume"},inplace=True)
    df.set_index("time",inplace=True)
    df = df[[f"{crypto}_close",f"{crypto}_volume"]]
    
    if len(master_df) == 0:
        master_df = df
    else:
        master_df = master_df.join(df)
    
master_df["future"] = master_df[f"{RATIO_PREDICT}_close"].shift(-FUT_PERIOD_PREDICT)

master_df["target"] = list(map(classify, master_df[f"{RATIO_PREDICT}_close"], master_df["future"]))

master_df[[f"{RATIO_PREDICT}_close", "future", "target"]].head(10)


,LTC-USD_close,future,target
time,,,
1528968660,96.580002,96.500000,0
1528968720,96.660004,96.389999,0
1528968780,96.570000,96.519997,0
1528968840,96.500000,96.440002,0
1528968900,96.389999,96.470001,1
1528968960,96.519997,96.400002,0
1528969020,96.440002,96.400002,0
1528969080,96.470001,96.400002,0
1528969140,96.400002,96.400002,0


In [5]:
times = sorted(master_df.index.values)
last_5pct_bound = times[-int(0.05*len(times))]

validation_master_df = master_df[(master_df.index >= last_5pct_bound)]
master_df = master_df[(master_df.index < last_5pct_bound)]


df_preprocess(master_df)
x_train, y_train = df_preprocess(master_df)
x_valid, y_valid = df_preprocess(validation_master_df)

print(f"train data: {len(x_train)} validation: {len(x_valid)}")
print(f"Buy_No: {y_train.count(0)} Buy_Yes: {y_train.count(1)}")
print(f"VALIDATION: Buy_No: {y_valid.count(0)} Buy_Yes: {y_valid.count(1)}")


train data: 69188 validation: 3062
Buy_No: 34594 Buy_Yes: 34594
VALIDATION: Buy_No: 1531 Buy_Yes: 1531


In [6]:
model = Sequential()
model.add(CuDNNLSTM(128, input_shape=(x_train.shape[1:]),return_sequences=True))
model.add(Dropout(0.2))
model.add(BatchNormalization())

model.add(CuDNNLSTM(128, input_shape=(x_train.shape[1:]),return_sequences=True))
model.add(Dropout(0.1))
model.add(BatchNormalization())

model.add(CuDNNLSTM(128, input_shape=(x_train.shape[1:])))
model.add(Dropout(0.2))
model.add(BatchNormalization())

model.add(Dense(32, activation="relu"))
model.add(Dropout(0.2))

model.add(Dense(2, activation="softmax"))

opt = tf.keras.optimizers.Adam(lr=0.001, decay=1e-6)

model.compile(loss="sparse_categorical_crossentropy",
             optimizer=opt,
             metrics=['accuracy'])

tensorboard = TensorBoard(log_dir = f'logs/{NAME}')

filepath = "RNN_Final-{epoch:02d}-{val_acc:.3f}"
checkpoint = ModelCheckpoint("models/{}.model".format(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max'))

history = model.fit(
    x_train,y_train,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    validation_data=(x_valid,y_valid),
    callbacks=[tensorboard,checkpoint])
              


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Train on 69188 samples, validate on 3062 samples
Instructions for updating:
Use tf.cast instead.


InvalidArgumentError: No OpKernel was registered to support Op 'CudnnRNN' used by node cu_dnnlstm/CudnnRNN (defined at <ipython-input-6-458b3e775ab7>:2) with these attrs: [is_training=true, seed2=0, input_mode="linear_input", T=DT_FLOAT, dropout=0, rnn_mode="lstm", direction="unidirectional", seed=0]
Registered devices: [CPU]
Registered kernels:
  <no registered kernels>

	 [[node cu_dnnlstm/CudnnRNN (defined at <ipython-input-6-458b3e775ab7>:2) ]]

Caused by op 'cu_dnnlstm/CudnnRNN', defined at:
  File "C:\Users\nairrohi\AppData\Local\conda\conda\envs\crypto-ml\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\nairrohi\AppData\Local\conda\conda\envs\crypto-ml\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\nairrohi\AppData\Local\conda\conda\envs\crypto-ml\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Users\nairrohi\AppData\Local\conda\conda\envs\crypto-m